In [37]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

#library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# Define a vector of package names to be used in the script
package_names.vec <- c(
  "tidyverse",   # Collection of data manipulation and visualization packages
  "ggtext",      # Add rich text elements to ggplot
  "showtext",    # Use custom fonts in plots
  "tidymodels",  # Framework for modeling and machine learning in R
  "furrr",       # For parallel computing
  "readxl",      # For reading excel files
  "tictoc",      # Performance measurement
  "glmnet"       # For LASSO and Elastic net models
)

# Define a function to install and load required packages
package_prep <- \(names.vec) {
# Iterate through each package name in the provided vector
for (name in names.vec) {
  # Check if the package is already installed
  if (!require(name, character.only = TRUE)) {
    # Install the package using renv if not already installed
    renv::install(name)
  }
}
}

# Pipe the vector of package names into the package_prep function
package_names.vec |> package_prep()

# Make the notebook work on both Kaggle and local computer
paths.list <- if(getwd() |> str_detect('kaggle')){
  # list of directories that are in kaggle workspace
  list(
    'zip path'=file.path('/kaggle','input','gfi-iran'),
    '2017 input'=file.path("/kaggle", "input", "gfi-iran", "micro_irn_varlabel.xls"),
    '2017 2 input'=file.path("/kaggle", "input", "gfi-iran", "micro_irn_varname.xls"),,
    '2021 input' = file.path("/kaggle", "input", "gfi-iran", "micro_irn.csv"),
    'plot path' = file.path('plots'),
    'analysis path' = file.path('analysis results')
  )
}else{
  # list of directories that are in local workspace
  list(
    'zip path'= file.path('data'),
    '2017 input'= file.path("intermediate-data","IRN_2017_FINDEX_v02_M_EXCEL","micro_irn_varlabel.xls"),
    '2017 2 input'=  file.path("intermediate-data","IRN_2017_FINDEX_v02_M_EXCEL","micro_irn_varname.xls"),
    '2021 input' = file.path("intermediate-data","micro_irn.csv"),
    'intermediate data' = file.path('intermediate-data'),
    'plot path' = file.path('plots'),
    'analysis path' = file.path('analysis results')
  )
}

# if it's on local computer, unzip the zipped world-bank data files
if(getwd() |> str_detect('[^kaggle]')){
list.files(
  paths.list |> pluck('zip path'),
  pattern = "\\.zip$",
  full.names = T
  ) |> 
  walk(~unzip(.x,exdir = paths.list |> pluck('intermediate data')))
}

# adjust the plot sizes to be big
options(repr.plot.width = 12, repr.plot.height = 8, repr.plot.res = 300)

options (pillar.print_max = 15) 
# adjust the tibble printing limit
options(pillar.width = 1000)

In [81]:
# Define a function to load and process a table
# Parameters:
# - name: Name of the table to be assigned in the global environment
# - path: Path to the file to be read
# - argyear: Year to be added as a column in the resulting table
# - reading_func: Function to read the file (e.g., read_csv, read_excel)
load_tbl <- function(name, path, argyear, reading_func) {
  # Read the file, process it, and store it in a variable 'result'
  result <- file.path(path) |>  # Generate the full file path
    reading_func() |>           # Read the file using the provided reading function
    mutate(
      year = argyear,           # Add a 'year' column with the specified year
      ID = row_number()         # Add an 'ID' column with sequential row numbers
    ) |> 
    select(where(~!all(is.na(.x)))) # Select only the columns that are not entirely NA
  
  # Assign the processed table to the specified name in the global environment
  assign(name, result, envir = .GlobalEnv)
}

# Call the 'load_tbl' function to load and process data for the year 2021
load_tbl("gfi_2021.tbl",               # Name of the output table
  paths.list |> pluck('2021 input'),  # File path
  2021,                               # Year to assign
  read_csv                            # Reading function
)

# Call the 'load_tbl' function to load and process data for the year 2017 (variable labels)
load_tbl("gfi_2017_1.tbl",            # Name of the output table
  paths.list |> pluck('2017 input'),  # File path
  2017,                               # Year to assign
  read_excel                          # Reading function
)

# Call the 'load_tbl' function to load and process data for the year 2017 (variable names)
load_tbl("gfi_2017_2.tbl",            # Name of the output table
  paths.list |> pluck('2017 2 input'),  # File path
  2017,                               # Year to assign
  read_excel                          # Reading function
)

# Replace spaces, colons, and hyphens in column names of `gfi_2017_1.tbl` with underscores.
new_names <- gsub("[ :-]", "_", gfi_2017_1.tbl |> names())

# Replace consecutive underscores (if any) with a single underscore in the column names.
new_names2 <- gsub("_+", "_", new_names)

# Rename the columns of `gfi_2017_1.tbl` using the cleaned-up column names.
gfi_2017_renamed.tbl <- gfi_2017_1.tbl |> rename(
  !!!setNames(
    names(gfi_2017_1.tbl),  # Current column names of the table.
    new_names2              # Updated column names after transformations.
  )
) |>
select(-c('Economy','Economy_Code','Gallup_World_Poll_identifier'))


# Define the named vector with keys in double quotes (replaced to become similar with 2017 column names.)
names_dict_2021 <- c(
"saved" = "Saved_in_the_past_year", #chosen
"borrowed" = "Borrowed_in_the_past_year", #chosen
"receive_wages" = "Received_wage_payments_in_past_12_months", #chosen
"receive_transfers" = "Received_a_government_transfer_payment",
"receive_pension" = "Received_a_government_pension_payment",
"pay_utilities" = "Payments_utility_bills",
"mobileowner" = "Owns_a_mobile_phone", #chosen
"internetaccess" = "Internet_access",
"anydigpayment" = "Made_or_received_a_digital_payment",
"fin34a" = "If_received_wage_payments_into_an_account", #chosen
"fin34b" = "If_received_wage_payments_through_a_mobile_phone", #chosen
"fin34d" = "If_received_wage_payments_in_cash", #chosen
"fin34e" = "If_received_wage_payments_to_a_card", #chosen
"fin37" = "Payments_government_transfers", #chosen
"fin38" = "Received_a_government_pension",
"fin39a" = "If_received_government_transfers_into_an_account", #chosen
"fin39b" = "If_received_government_transfers_through_a_mobile_phone", #chosen
"fin39d" = "If_received_government_transfers_in_cash", #chosen
"fin39e" = "If_received_government_transfers_to_a_card", #chosen
"fin44a" = "Financially_worried_old_age",
"fin44b" = "Financially_worried_medical_cost",
"fin44c" = "Financially_worried_bills",
"fin44d" = "Financially_worried_education",
"fin45" = "Financially_most_worried",
"fin16" = "Saved_in_past_12_months_for_old_age",
"fin17a" = "Saved_in_past_12_months_using_an_account_at_a_financial_institution",
"fin17a1" = "Saved_using_a_mobile_money_account",
"fin20" = "Borrowed_for_medical_purposes",
"fin22a" = "Borrowed_from_a_financial_institution",
"fin22b" = "Borrowed_from_family_or_friends",
"fin24" = "Main_source_of_emergency_funds",
"fin24a" = "Difficulty_of_emergency_funds_in_30_days",
"fin24b" = "Difficulty_of_emergency_funds_in_7_days",
"fin30" = "Paid_utility_bills_in_past_12_months", #chosen
"fin31a" = "If_paid_utility_bills_using_an_account", #chose
"fin31b" = "If_paid_utility_bills_through_a_mobile_phone", #chosen
"fin31c" = "If_paid_utility_bills_in_cash", #chosen
"fin32" = "Payments_wage_payments",
"fin33" = "Received_public_sector_wage_payments",
"fin11b" = "If_does_not_have_account_b/c_too_expensive", #chosen
"fin11c" = "If_does_not_have_account_b/c_lack_documentation", #chosen
"fin11d" = "If_does_not_have_account_b/c_lack_trust", #chosen
"fin11e" = "If_does_not_have_account_b/c_religious_reasons", #chosen
"fin11f" = "If_does_not_have_account_b/c_lack_of_money", #chosen
"fin11g" = "If_does_not_have_account_b/c_family_member_already_has_one", #chosen
"fin11h" = "If_does_not_have_account_b/c_no_need_for_financial_services", #chosen
"fin13a" = "Use_mobile_money_account_two_or_more_times_a_month",
"fin13b" = "Use_mobile_money_account_to_store_money",
"fin13c" = "Use_mobile_money_account_to_borrow_money",
"fin13d" = "Use_mobile_money_account_without_help",
"fin14_1" = "Use_mobile_phone_to_pay_for_a_purchase_in_store",
"fin14a" = "Made_bill_payments_online_using_the_Internet",
"fin14a1" = "Send_money_to_a_relative_or_friend_online_using_the_Internet",
"fin14b" = "Bought_something_online_using_the_Internet",
"account_mob" = "Has_a_mobile_money_account", #chosen
"fin2" = "Has_a_debit_card",
"fin4" = "Used_a_debit_card",
"fin5" = "Used_a_mobile_phone_or_internet_to_access_account",
"fin6" = "Used_a_mobile_phone_or_internet_to_check_account_balance",
"fin7" = "Has_a_credit_card", #chosen
"fin8" = "If_has_credit_card_used_card_in_past_12_months", #chosen
"fin8b" = "Paid_credit_card_balances_in_full",
"fin9" = "If_has_account_any_deposit_into_account_in_past_12_months", #chosen
"fin9a" = "Make_deposits_into_the_account_two_or_more_times_per_month",
"fin10" = "If_has_account_any_withdrawal_from_account_in_past_12_months", #chosen
"fin10a" = "Withdrew_from_the_account_two_or_more_times_per_month",
"fin10b" = "Used_account_to_store_money",
"fin11_1" = "Unbanked_use_account_without_help",
"fin11a" = "If_does_not_have_account_b/c_too_far_away", #chosen
"economy" = "Economy", #omitted
"economycode" = "Economy_Code", #omited
"wpid_random" = "Gallup_World_Poll_identifier", #omitted
"wgt" = "Weight", #chosen
"female" = "Respondent_is_female", #chosen
"age" = "Respondent_age", #chosen
"educ" = "Respondent_education_level", #chosen
"inc_q" = "Within_economy_household_income_quintile", #chosen
"emp_in" = "Respondent_is_in_the_workforce", #chosen
"account" = "Has_an_account", #chosen
"account_fin" = "Has_an_account_at_a_financial_institution", #chosen
"year" = "year", #chosen
"ID" = "ID" # chosen
)
vars.vec <- c('Respondent_age','Has_an_account_at_a_financial_institution',
           'Has_an_account','Has_a_mobile_money_account','Weight','ID','Respondent_is_female',
           'Respondent_education_level','Respondent_is_in_the_workforce','year',
           'Within_economy_household_income_quintile',"Owns_a_mobile_phone","Saved_in_the_past_year",
            "If_has_account_any_deposit_into_account_in_past_12_months", 'Borrowed_in_the_past_year',
              "If_has_account_any_withdrawal_from_account_in_past_12_months",'If_has_credit_card_used_card_in_past_12_months', 
                "Received_wage_payments_in_past_12_months",'If_does_not_have_account_b/c_too_expensive',
                'If_does_not_have_account_b/c_too_far_away','Has_a_credit_card','Payments_government_transfers',
                               'If_does_not_have_account_b/c_lack_documentation',"If_paid_utility_bills_using_an_account",
                               'If_does_not_have_account_b/c_lack_trust',
                               'If_does_not_have_account_b/c_religious_reasons',
                               'If_does_not_have_account_b/c_lack_of_money','Paid_utility_bills_in_past_12_months',
                               'If_does_not_have_account_b/c_family_member_already_has_one',
                               "If_paid_utility_bills_through_a_mobile_phone",'If_paid_utility_bills_in_cash',
                               'If_does_not_have_account_b/c_no_need_for_financial_services',"If_received_wage_payments_into_an_account","If_received_wage_payments_through_a_mobile_phone","If_received_wage_payments_in_cash","If_received_wage_payments_to_a_card",
                              "If_received_government_transfers_into_an_account", "If_received_government_transfers_through_a_mobile_phone","If_received_government_transfers_in_cash","If_received_government_transfers_to_a_card",
                            "Saved_in_past_12_months_for_old_age","Saved_in_past_12_months_using_an_account_at_a_financial_institution"    )

gfi_2021_renamed.tbl  <- gfi_2021.tbl |> 
  rename(
    !!!setNames(
      names(gfi_2021.tbl), 
      names_dict_2021[names(gfi_2021.tbl)]
    )
) |> 
select(-c('Economy','Economy_Code','Gallup_World_Poll_identifier'))

Rows: 1005 Columns: 84
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): economy, economycode
dbl (78): wpid_random, wgt, female, age, educ, inc_q, emp_in, account, accou...
lgl  (4): urbanicity_f2f, receive_agriculture, remittances, merchantpay_dig

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [82]:
gfi_2017_renamed.tbl |> select(-all_of(vars.vec)) |> names()

 [1] "Has_a_debit_card"                                                    
 [2] "If_has_debit_card_card_in_own_name"                                  
 [3] "If_has_debit_card_used_card_in_past_12_months"                       
 [4] "Used_mobile_phone_or_internet_to_access_FI_account"                  
 [5] "Used_mobile_phone_or_internet_to_check_account_balance"              
 [6] "Made_bill_payments_online_using_the_Internet"                        
 [7] "Bought_something_online_using_the_Internet"                          
 [8] "Saved_in_past_12_months_for_farm/business_purposes"                  
 [9] "Saved_in_past_12_months_for_old_age"                                 
[10] "Saved_in_past_12_months_using_an_account_at_a_financial_institution" 
[11] "Has_loan_from_a_financial_institution_for_home,_apartment,_or_land"  
[12] "Borrowed_in_past_12_months_for_medical_purposes"                     
[13] "Borrowed_in_past_12_months_for_farm/business_purposes"               
[14] "Borrow

In [83]:
non_binary_vars.vec <- c('Respondent_age','Weight','ID','Respondent_is_female','Respondent_education_level',
                         'Respondent_is_in_the_workforce','year','Within_economy_household_income_quintile')

binary_vars.vec <- vars.vec[!vars.vec %in% non_binary_vars.vec]


coded.tbl <- gfi_2017_renamed.tbl |> select(all_of(vars.vec)) |> 
  
# Mutate to clean and standardize categorical variables
  mutate(
    Respondent_education_level = case_match(
      Respondent_education_level,
      "completed primary or less" ~ 0,
      "secondary" ~ 1,
      "completed tertiary or more" ~ 2,
      .default = NA_integer_  # Replace "don't know" or "refused" responses with NA
    ),
    Respondent_is_in_the_workforce = case_match(
      Respondent_is_in_the_workforce,
      'in workforce' ~ 1,
      'out of workforce' ~ 0,
      .default = NA_integer_
    ),
    Within_economy_household_income_quintile = case_match(
      Within_economy_household_income_quintile,
      "Poorest 20%" ~ 0,
      "Second 20%" ~ 1,
      "Middle 20%" ~ 2,
      "Fourth 20%" ~ 3,
      "Richest 20%" ~ 4,
      .default = NA_integer_
    ),
    Respondent_is_female = case_match(
      Respondent_is_female,
      'Male' ~ 0,
      "Female" ~ 1,
      .default = NA_integer_
    ),
    Respondent_age = case_when(
      Respondent_age == "99+" ~ "100", # Convert "99+" to a numeric value
      .default = Respondent_age
    ) |> as.integer(),
    year = 0,
    across(
        all_of(binary_vars.vec),
        ~case_match(
            .x,
            "yes" ~ 1,
            "0" ~ 0,
            "no" ~ 0,
            .default = NA_integer_
        )
    )      
) |> 
  bind_rows(
      gfi_2021_renamed.tbl |> select(all_of(vars.vec))  |> 
    # Mutate to clean and standardize categorical variables using case_match.
    mutate(
      # Map numeric values of Respondent_is_female to meaningful labels.
      Respondent_is_female = if_else(Respondent_is_female==2,0,Respondent_is_female),
      # Map education level codes to descriptive labels.
      Respondent_education_level = Respondent_education_level - 1,
      # Map workforce participation status to descriptive labels.
      Respondent_is_in_the_workforce = if_else(Respondent_is_in_the_workforce==2,0,Respondent_is_in_the_workforce),
      # Map income quintiles to descriptive labels for within-economy household income.
      Within_economy_household_income_quintile = Within_economy_household_income_quintile-1,
        Received_wage_payments_in_past_12_months = case_match(
            Received_wage_payments_in_past_12_months,
            c(1,2,3) ~ 1,
            4 ~ 0,
            .default = NA
        ),
        year= 1,
        across(
            all_of(binary_vars.vec),
            ~case_match(
                .x,
                1 ~ 1,
                c(0,2) ~ 0,
                .default = NA
            )
        )
    )
  ) |> 
  mutate(
    across(
      -Weight,
    ~as.integer(.x)
    )
  )|> rename(
    is2021 = year,
    phone = Owns_a_mobile_phone,
    saved = Saved_in_the_past_year,
    female = Respondent_is_female,
    in_wf = Respondent_is_in_the_workforce,
    inc_q = Within_economy_household_income_quintile,
    educ = Respondent_education_level,
    age = Respondent_age,
    fin_acc = Has_an_account_at_a_financial_institution,
    acc = Has_an_account,
    mm_acc = Has_a_mobile_money_account,
    wgt = Weight,
    id = ID,
    depos = If_has_account_any_deposit_into_account_in_past_12_months, 
    withd = If_has_account_any_withdrawal_from_account_in_past_12_months, 
    wage = Received_wage_payments_in_past_12_months,
    nofin_exp = `If_does_not_have_account_b/c_too_expensive`,
    nofin_doc = `If_does_not_have_account_b/c_lack_documentation`,
    nofin_trst = `If_does_not_have_account_b/c_lack_trust`,
    nofin_rlg = `If_does_not_have_account_b/c_religious_reasons`,
    nofin_pov = `If_does_not_have_account_b/c_lack_of_money`,
    nofin_fam = `If_does_not_have_account_b/c_family_member_already_has_one`,
    nofin_use = `If_does_not_have_account_b/c_no_need_for_financial_services`,
    nofin_far = `If_does_not_have_account_b/c_too_far_away`,
    wag_acc = If_received_wage_payments_into_an_account,
    wag_mob = If_received_wage_payments_through_a_mobile_phone,
    wag_csh = If_received_wage_payments_in_cash,
    wag_crd = If_received_wage_payments_to_a_card,
    cred_crd = Has_a_credit_card,
    util = Paid_utility_bills_in_past_12_months,
    borrow = Borrowed_in_the_past_year,
    use_cc = If_has_credit_card_used_card_in_past_12_months,
    trnsf = Payments_government_transfers,
    util_acc = If_paid_utility_bills_using_an_account,
    util_ph = If_paid_utility_bills_through_a_mobile_phone,
    util_csh = If_paid_utility_bills_in_cash,
    trns_acc = If_received_government_transfers_into_an_account,
    trns_ph = If_received_government_transfers_through_a_mobile_phone,            
    trns_csh = If_received_government_transfers_in_cash,
    trns_crd = If_received_government_transfers_to_a_card,
    save_old = Saved_in_past_12_months_for_old_age,
    save_acc = Saved_in_past_12_months_using_an_account_at_a_financial_institution
)
coded.tbl

# A tibble: 2,009 × 40
     age fin_acc   acc mm_acc   wgt    id female  educ in_wf is2021 inc_q phone
   <int>   <int> <int>  <int> <dbl> <int>  <int> <int> <int>  <int> <int> <int>
 1    19       0     1      1 2.17      1      0     1     0      0     1     1
 2    42       1     1      0 0.674     2      1     0     0      0     0     1
 3    36       1     1      1 0.295     3      0     0     1      0     2     1
 4    35       1     1      0 0.932     4      1     1     0      0     2     1
 5    22       1     1      0 1.42      5      0     1     1      0     1     1
 6    55       1     1      0 0.286     6      0     2     1      0     1     1
 7    49       1     1      0 0.289     7      0     0     1      0     0     1
 8    37       1     1      0 1.25      8      1     0     0      0     2     0
 9    20       1     1      0 1.92      9      1     1     0      0     2     0
10    23       1     1      1 0.466    10      0     2     1      0     4     1
   saved depos bo

In [ ]:

sliced.tbl <- coded.tbl |> 
  mutate(wgt_n = wgt / sum(wgt)) |>      # Create a column with weights normalized to sum to 1
  slice_sample(                                     # Perform weighted random sampling on the dataset
    n = 1e6,                                       # Number of samples to draw (1 million)
    weight_by = wgt_n,                        # Weights column used for sampling probabilities
    replace = TRUE                                 # Allow sampling with replacement
  )

$Borrowed_in_the_past_year
[1] "yes" "0"  



In [72]:
gfi_2021_renamed.tbl |> select("Borrowed_in_the_past_year") |> map(~unique(.x)) |> print()

$Borrowed_in_the_past_year
[1] 1 0

